https://www.restack.io/p/telugu-speech-to-text-answer-converter-cat-ai

In [22]:
from transformers import pipeline

# Load the Speech2Text pipeline for Telugu
speech_to_text = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-xlsr-53")

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [ ]:
# Load your audio file
result = speech_to_text("../../Speech-Data/pilot/ID_4_PILOT/ID_4_level2_గాటు.wav")

# Output the transcription
print(result["text"])  # This will print the converted Telugu text

wavetovec

https://www.restack.io/p/speech-to-text-answer-converter-software-cat-ai#cm23l76043loo9srqmfxax2eu

In [17]:
from transformers import Wav2Vec2Model
import torch
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)

ImportError: 
Wav2Vec2Model requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [15]:
import torch
x = torch.rand(5, 3)
print(x)


tensor([[0.6752, 0.7642, 0.1534],
        [0.3115, 0.3887, 0.4980],
        [0.4537, 0.7550, 0.1625],
        [0.0863, 0.0384, 0.7614],
        [0.5362, 0.1685, 0.9533]])


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-960h')
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

In [ ]:
import torchaudio

waveform, sample_rate = torchaudio.load('../../Speech-Data/pilot/ID_4_PILOT/ID_4_level2_గాటు.wav')

# Resample if necessary
if sample_rate != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

In [ ]:
input_values = processor(waveform.squeeze().numpy(), return_tensors='pt', padding='longest').input_values

# Retrieve logits
with torch.no_grad():
    logits = model(input_values).logits

# Get predicted ids
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the ids to text
transcription = processor.batch_decode(predicted_ids)
print(transcription)

wavetovec

https://www.kdnuggets.com/2021/03/speech-text-wav2vec.html

In [18]:
import librosa

#Importing Pytorch
import torch

#Importing Wav2Vec
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [19]:
audio, rate = librosa.load("../../Speech-Data/pilot/ID_4_PILOT/ID_4_level2_గాటు.wav", sr = 16000)

In [20]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/keerthi/miniconda3/envs/speechenv/lib/python3.9/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


ImportError: 
Wav2Vec2ForCTC requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
input_values = tokenizer(audio, return_tensors = "pt").input_values
logits = model(input_values).logits
prediction = torch.argmax(logits, dim = -1)
transcription = tokenizer.batch_decode(prediction)[0]

In [ ]:
print(transcription)

https://huggingface.co/docs/transformers/en/model_doc/wav2vec2

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch

# Import model, feature extractor, and tokenizer for Telugu ASR
model = AutoModelForCTC.from_pretrained("ai4bharat/indicwav2vec-te")
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicwav2vec-te")
feature_extractor = AutoFeatureExtractor.from_pretrained("ai4bharat/indicwav2vec-te")

# Load first sample of Telugu Common Voice or a relevant Telugu dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "te", split="train", streaming=True, trust_remote_code=True)
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=16_000))
dataset_iter = iter(dataset)
sample = next(dataset_iter)

# Forward sample through model to get greedily predicted transcription ids
input_values = feature_extractor(sample["audio"]["array"], return_tensors="pt").input_values
logits = model(input_values).logits[0]
pred_ids = torch.argmax(logits, axis=-1)

# Retrieve word stamps (analogous commands for `output_char_offsets`)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
# Compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in outputs.word_offsets
]

# Display the first few word offsets
word_offsets[:3]
